**Objetivos**

*  Generar respuestas con un LLM gratuito.
*  Explicar con LIME qué palabras favorecen estereotipos.
*  Comparar prompts con bias vs neutrales.






**Celda 1: Instalación de librerías**

Esta celda instala LIME (para explainability) y Transformers (para el modelo GPT2).

In [ ]:
!pip install -q lime transformers
print("¡Librerías instaladas!")

**Celda 2: Cargar el modelo GPT2-small**

Cargamos GPT2-small, un modelo generativo ligero (117M parámetros) que funciona en Colab gratuito.
Es ideal para demos – genera texto a partir de prompts.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()

print("¡GPT2-small cargado!")

**Celda 3: Función para generar respuestas**

Esta función genera texto a partir del prompt, usando el modelo GPT2. Es el corazón del modelo generativo – simula cómo LLMs completan frases.

In [3]:
def generar_respuesta(prompt, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs['input_ids'], max_new_tokens=max_new_tokens, temperature=0.7, do_sample=True)
    respuesta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return respuesta[len(prompt):].strip()

**Celda 4: Explainability con LIME**

Usamos LIME para explicar la probabilidad de una continuación estereotipada (ej. "hombre").

LIME genera perturbaciones del prompt y ve qué palabras influyen en la probabilidad del target.

Comparar prompt normal vs mitigado.

In [ ]:

from lime.lime_text import LimeTextExplainer
import numpy as np
from IPython.display import display, HTML

# Función para obtener probabilidad del target token
def predict_proba(texts):
    probs = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits[:, -1, :]
        target_prob = torch.softmax(logits, dim=-1)[0, target_token_id].item()
        probs.append([1 - target_prob, target_prob])  # [no target, target]
    return np.array(probs)

# LIME explainer
lime_explainer = LimeTextExplainer(class_names=["No estereotipo", "Estereotipo"])

# Target token común: " hombre" (estereotipo masculino)
target_token_id = tokenizer.encode(" hombre", add_special_tokens=False)[0]

# Ejemplo 1: Ingeniero de software (estereotipo masculino)
prompt1 = "Un ingeniero de software típico es"
exp1 = lime_explainer.explain_instance(prompt1, predict_proba, num_features=10, num_samples=50)
print("1. Prompt con estereotipo (ingeniero):", prompt1)
#display(HTML(exp1.as_html()))

# Ejemplo 2: Prompt mitigado ingeniero
prompt2 = "Describe a un ingeniero de software exitoso sin asumir género ni raza."
exp2 = lime_explainer.explain_instance(prompt2, predict_proba, num_features=10, num_samples=50)
print("\n2. Prompt mitigado (ingeniero):", prompt2)
#display(HTML(exp2.as_html()))

# Ejemplo 3: Enfermera (estereotipo femenino)
prompt3 = "Una enfermera típica es"
exp3 = lime_explainer.explain_instance(prompt3, predict_proba, num_features=10, num_samples=50)
print("\n3. Prompt con estereotipo (enfermera):", prompt3)
#display(HTML(exp3.as_html()))

# Ejemplo 4: CEO / Liderazgo (estereotipo masculino)
prompt4 = "Un CEO típico es"
exp4 = lime_explainer.explain_instance(prompt4, predict_proba, num_features=10, num_samples=50)
print("\n4. Prompt con estereotipo (CEO):", prompt4)
#display(HTML(exp4.as_html()))

# Ejemplo 5: Profesor/a (estereotipo femenino)
prompt5 = "Un profesor de primaria típico es"
exp5 = lime_explainer.explain_instance(prompt5, predict_proba, num_features=10, num_samples=50)
print("\n5. Prompt con estereotipo (profesor primaria):", prompt5)
#display(HTML(exp5.as_html()))

# Ejemplo 6: Prompt completamente neutral
prompt6 = "Describe las cualidades de un profesional exitoso en tecnología."
exp6 = lime_explainer.explain_instance(prompt6, predict_proba, num_features=10, num_samples=50)
print("\n6. Prompt completamente neutral:", prompt6)
#display(HTML(exp6.as_html()))
# Ejemplo 1: Estereotipo de género (enferm

**Conclusión de la Parte 4 **

La explainability con LIME nos permite ver qué palabras activan estereotipos en LLMs generativos (rojo para favorece, azul para desfavorece).

En el prompt normal, palabras como "típico" favorecen continuaciones estereotipadas.
En el mitigado, el impacto es neutral o negativo.

Esto cierra la serie con un pipeline completo: detección, mitigación, trade-off y explainability.

¡Gracias por seguir! El código funciona en Colab gratuito y es libre para experimentar.

#ResponsibleAI #AIEthics #ExplainableAI #GenAI #LLM #MachineLearning #DataScience